# Quantum System: QSVM

This code outlines the implementation of a QSVM which is used to classify data speccifalcy classify linerally seperable data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

algorithm_globals.random_seed = 42

In [2]:
!pip install qiskit_machine_learning

  Using cached qiskit_machine_learning-0.7.2-py3-none-any.whl.metadata (12 kB)
  Using cached fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl
Using cached qiskit_machine_learning-0.7.2-py3-none-any.whl (97 kB)


In [2]:
#importing for data formatting and representation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
dataset = pd.read_csv('Social_Network_Ads.csv')# data set used for the entire project
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

## Trainning Data

Much like in a classical system, a quantum system uses a smaller subset of data to train on and in this case we are using 25% of the original data set. This allows for the most effecient learning and optimization time possible while still trying to maintain accuarcy.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print(X_train[:4])
print(y_train[:4])
print(X_test[:4])
print(y_test[:4])

[[    44  39000]
 [    32 120000]
 [    38  50000]
 [    32 135000]]
[0 1 0 1]
[[   30 87000]
 [   38 50000]
 [   35 75000]
 [   30 79000]]
[0 0 0 0]


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement="linear")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

In [12]:
from qiskit_machine_learning.algorithms import QSVC

qsvc = QSVC(quantum_kernel=kernel)

qsvc.fit(X_train, y_train)

qsvc_score = qsvc.score(X_test, y_test)

print(f"QSVC classification test score: {qsvc_score}")

QSVC classification test score: 0.61
